In [1]:
import pandas as pd
import duckdb
import psycopg2

In [2]:
with open('db_password.txt', 'r') as file:
    db_password = file.read().strip()

psql_conn = psycopg2.connect("dbname = 'DB_FP' user = 'postgres' host = 'localhost' password = " + db_password)

table_names = ['item', 'lost_item', 'found_item', 'users', 'locations', 'locates', 'stores', 'returns_or_claims', 'category', 'belongs', 'posts', 'members', 'comments', 'message', 'feedback', 'notifies', 'reward'] 
con = duckdb.connect()

for table_name in table_names:
    query_str = "SELECT * FROM " + table_name
    df = pd.read_sql_query(query_str, psql_conn)
    con.register(table_name, df)

psql_conn.close()

/var/folders/y6/fdlf2f113hdd9mwtf7h1m_2r0000gn/T/ipykernel_7546/2643837570.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, psql_conn)
/var/folders/y6/fdlf2f113hdd9mwtf7h1m_2r0000gn/T/ipykernel_7546/2643837570.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, psql_conn)
/var/folders/y6/fdlf2f113hdd9mwtf7h1m_2r0000gn/T/ipykernel_7546/2643837570.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, psql_conn)
/var/folde

In [3]:
# function that sees SQL command AS input and output table from database
def query(query):
    result = con.execute(query).fetchall()
    column_names = [desc[0] for desc in con.description]
    return column_names, result

In [4]:
# Query row count for each table and store in DataFrame
row_counts = []
for table_name in table_names:
    query_str = f"SELECT COUNT(*) FROM {table_name}"
    result = con.execute(query_str).fetchall()[0][0]
    row_counts.append({'Table Name': table_name, 'Row Count': result})

# Convert to DataFrame
row_counts_df = pd.DataFrame(row_counts)

row_counts_df

,Table Name,Row Count
0,item,3000
1,lost_item,1493
2,found_item,1507
3,users,5000
4,locations,47
5,locates,3000
6,stores,3000
7,returns_or_claims,10000
8,category,10
9,belongs,3000


In [5]:
columns, data = query('''
SELECT m.senderid, u_s.username as senderName, m.receiverid, u_r.username as receiverName, m.mgtime, mgcontent
from MESSAGE as m
join USERS as u_s on m.senderid = u_s.userid
join USERS as u_r on m.receiverid = u_r.userid
          ''')
message = pd.DataFrame(data, columns=columns)
message

,senderid,senderName,receiverid,receiverName,mgtime,mgcontent
0,US00000046,孫昌芬,US00004198,莫蕙海,2024-11-05 01:07:15,感謝好心人找到了我的東西！
1,US00000047,皇甫達舟,US00001407,朱玉盈,2024-11-08 08:04:49,我丟了它請問在哪裡找回來？
2,US00000118,龔達怡,US00000963,陳可俞,2024-11-17 09:50:42,我記得放在椅子上，請問可以再提供一些細節嗎？
3,US00000272,曹詩可,US00001437,崔思婷,2024-11-14 10:34:43,真的很感謝，這個物品對我很重要！
4,US00000282,龔榆靈,US00000330,馬建茜,2024-11-09 01:54:23,謝謝你，我終於找回來了！
...,...,...,...,...,...,...
95,US00004959,毛新宗,US00000233,程惠家,2024-11-27 08:40:37,感謝好心人找到了我的東西！
96,US00004967,阮壬宜,US00003873,滕蓉達,2024-11-09 20:34:30,感謝好心人找到了我的東西！
97,US00004730,王壬淑,US00003946,沈沛鐸,2024-11-28 19:23:50,感謝找到我的筆記本，已經取回了！
98,US00004959,毛新宗,US00002349,冉暘昌,2024-11-13 15:12:28,謝謝你，我終於找回來了！


In [6]:
def message(sender, receiver):
    columns, data = query(f'''
    SELECT m.senderid, u_s.username as senderName, m.receiverid, u_r.username as receiverName, m.mgtime, mgcontent
    from MESSAGE as m
    join USERS as u_s on m.senderid = u_s.userid
    join USERS as u_r on m.receiverid = u_r.userid
    where senderName = '{sender}' and receiverName = '{receiver}'
            ''') 
    return pd.DataFrame(data, columns=columns)

sender = "毛新宗"
receiver = "程惠家"
message(sender, receiver)

,senderid,senderName,receiverid,receiverName,mgtime,mgcontent
0,US00004959,毛新宗,US00000233,程惠家,2024-11-27 08:40:37,感謝好心人找到了我的東西！


In [7]:
def feedback():
    columns, data = query('''
    SELECT f.userid, u.username, f.fbtime, f.fbcontent
    from feedback as f
    join USERS as u on f.userid = u.userid
    ''')
    return pd.DataFrame(data, columns=columns).sort_values("fbtime")

feedback()

,userid,username,fbtime,fbcontent
414,US00004521,冉偉海,2023-01-02 15:58:23,希望能夠增加一個自動通知功能，當有相關物品被找到時，系統能及時通知我。
449,US00004475,張光俞,2023-01-03 05:16:40,真的非常感謝這個平台，我找回了丟失的背包，這對我來說很重要。已經推薦給我的朋友們了！
140,US00000190,郭豪婷,2023-01-03 07:01:22,希望能夠增加一個自動通知功能，當有相關物品被找到時，系統能及時通知我。
386,US00004234,程志心,2023-01-03 14:14:06,這個系統太棒了！我丟失的耳機找回來了，非常感謝！
57,US00000804,陳慧玉,2023-01-05 00:17:28,希望能在物品描述中添加照片的上傳功能，這樣能幫助我更快地找到丟失的物品。
...,...,...,...,...
197,US00002266,司馬翰宇,2024-11-22 11:51:26,最近在提交失物信息時，系統有時會顯示錯誤，無法成功提交，請協助處理一下。
9,US00000145,余達韋,2024-11-25 22:50:58,最近在提交失物信息時，系統有時會顯示錯誤，無法成功提交，請協助處理一下。
199,US00002303,蘇茜宏,2024-11-26 01:16:49,希望能夠增加一個自動通知功能，當有相關物品被找到時，系統能及時通知我。
309,US00003898,林佳君,2024-11-28 04:08:06,最近在提交失物信息時，系統有時會顯示錯誤，無法成功提交，請協助處理一下。


In [8]:
columns, data = query('''
SELECT c.memberid, u.username, c.itemid, i.description, c.cmtime, c.cmcontent
from COMMENTS as c
join USERS as u on c.memberid = u.userid
join ITEM as i on c.itemid = i.itemid
''')
# 將結果轉換為 DataFrame
comment = pd.DataFrame(data, columns=columns)
comment

,memberid,username,itemid,description,cmtime,cmcontent
0,US00004444,尤惠俊,IT00000008,高科技的手鏈,2024-11-08 07:54:27,感謝找到我的筆記本，已經取回了！
1,US00004409,盧暘欣,IT00000010,輕便的包包,2024-11-19 01:08:22,我的丟失了，請問這是我的嗎？
2,US00004833,林宇新,IT00000012,小巧的枕頭,2024-11-16 04:35:25,感謝好心人找到了我的東西！
3,US00003084,秦欣靈,IT00000015,手工的帽子,2024-11-15 01:09:03,非常感謝，你讓我找回了我的東西，我會盡快來領取！
4,US00003532,武靜葦,IT00000019,綠色的鑰匙,2024-11-18 01:48:10,謝謝你撿到！真的太感謝了！
...,...,...,...,...,...,...
995,US00001503,嚴婷達,IT00002456,綠色的地圖,2024-11-19 15:28:59,真的很感謝，這個物品對我很重要！
996,US00003040,歐陽妍建,IT00002536,輕便的行李箱,2024-11-10 04:44:26,我弄丟了它，請問可以提供更詳細的描述嗎？
997,US00004893,邱蕙修,IT00002702,小巧的圍巾,2024-11-28 01:11:56,謝謝你撿到我的耳機，我已經領回來了。
998,US00003076,陳艾凌,IT00002993,便攜的帽子,2024-11-22 17:00:54,謝謝你撿到我的物品！我會去指定地點領取。


In [9]:
def comment(itemID):
    columns, data = query(f'''
    SELECT c.memberid, u.username, c.itemid, i.description, c.cmtime, c.cmcontent
    from COMMENTS as c
    join USERS as u on c.memberid = u.userid
    join ITEM as i on c.itemid = i.itemid
    WHERE i.itemid = '{itemID}'
    ''')
    # 將結果轉換為 DataFrame
    return pd.DataFrame(data, columns=columns).sort_values("cmtime")

itemID = 'IT00001506'
comment(itemID)

,memberid,username,itemid,description,cmtime,cmcontent
1,US00004937,程澄熙,IT00001506,粉色的地圖,2024-11-11 16:57:25,謝謝你撿到！真的太感謝了！
0,US00002811,田涵淑,IT00001506,粉色的地圖,2024-11-15 19:48:32,我的丟失了，請問這是我的嗎？
